In [1]:
#讀檔
import json

text = []
for i in range(65,78):
    range_2 = 10
    if i == 77:
        range_2 = 8
    for j in range(range_2):
        range_3 = 10
        if i == 77 and j == 7:
            range_3 = 4
        for l in range(range_3):
            path = 'wiki_zh/A' + chr(i) + '/wiki_' + str(j) + str(l)

            input_file = open(path, encoding = 'utf8')
            for line in input_file.readlines():
                text.append(json.loads(line))
            input_file.close()
            

In [2]:
#簡轉繁
from opencc import OpenCC

cc = OpenCC('s2t')

for item in text:
    item['title'] = cc.convert(item['title'])
    item['text'] = cc.convert(item['text'])

In [3]:
#斷詞
import jieba

fileTrainSeg=[]
for i in text:
    fileTrainSeg.append([' '.join(list(jieba.cut(i['text'],cut_all=False)))])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\UserTest\AppData\Local\Temp\jieba.cache
Loading model cost 1.229 seconds.
Prefix dict has been built successfully.


In [4]:
#斷詞結果寫檔
fileSegWordDonePath ='corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0].encode('utf-8'))
        fW.write('\n'.encode('utf-8'))

In [5]:
#轉word2vec向量
import word2vec

word2vec.word2vec('corpusSegDone.txt', 'corpusWord2Vec.txt', size = 300, verbose = True)

Running command: word2vec -train corpusSegDone.txt -output corpusWord2Vec.txt -size 300 -window 5 -sample 1e-3 -hs 0 -negative 5 -threads 12 -iter 5 -min-count 5 -alpha 0.025 -debug 2 -binary 0 -cbow 1
Starting training using file corpusSegDone.txt
Vocab size: 9233138800K0KK
Words in train file: 244600906


Alpha: 0.011181  Progress: 55.28%  Words/thread/sec: 146.68k  

Alpha: 0.000021  Progress: 99.92%  Words/thread/sec: 143.63k   0.011166  Progress: 55.34%  Words/thread/sec: 146.66k  0.011045  Progress: 55.83%  Words/thread/sec: 147.19k  19  Progress: 55.93%  Words/thread/sec: 147.27k  a: 0.010984  Progress: 56.07%  Words/thread/sec: 147.45k    Progress: 56.44%  Words/thread/sec: 147.92k  41.45k  1%  Words/thread/sec: 141.42k  hread/sec: 141.46k  s/thread/sec: 141.64k  d/sec: 141.97k  Progress: 58.33%  Words/thread/sec: 142.20k  2.58k  Words/thread/sec: 142.87k   58.98%  Words/thread/sec: 142.91k  thread/sec: 143.09k  s: 59.33%  Words/thread/sec: 143.12k  ha: 0.010157  Progress: 59.38%  Words/thread/sec: 143.17k  ad/sec: 143.18k  /thread/sec: 143.39k  64  Progress: 59.75%  Words/thread/sec: 143.57k   143.85k  .009961  Progress: 60.16%  Words/thread/sec: 143.99k  rogress: 60.21%  Words/thread/sec: 144.06k  9919  Progress: 60.33%  Words/thread/sec: 144.20k   60.95%  Words/thread/sec: 144.97k  ords/thread/sec: 145.03k  45.10k  a: 0.009639  Progress: 61

In [6]:
#載入模組
model = word2vec.load('corpusWord2Vec.txt')

In [13]:
#計算相似度
import numpy as np

word = '日本'
n = 20

metrics = np.dot(model.vectors, model[word].T)
best = np.argsort(metrics)[::-1][1:n+1]

In [14]:
#輸出結果
num = 1
for index in best:
    print(num, '. ', model.vocab[index])
    num = num + 1

1 .  日本政府
2 .  伊豆
3 .  廣島
4 .  福岡縣
5 .  香川
6 .  韓國
7 .  和歌山
8 .  京都
9 .  山梨
10 .  奈良
11 .  北九州
12 .  千葉縣
13 .  九州
14 .  近畿
15 .  兵庫
16 .  愛媛縣
17 .  東京帝
18 .  土佐
19 .  關西
20 .  靜岡縣
